In [3]:
import jsonlines
import itertools
import pandas as pd
from pprint import pprint
import datasets
import transformers
from datasets import load_dataset
from lamini import Lamini
import os
import lamini

In [4]:
lamini.api_key = "c6a1b973a6abc01cffe698e4a85e5e3027b60ce192c093a0c060a8d2d8e5303b"


In [6]:
instruction_tuned_dataset = load_dataset("tatsu-lab/alpaca", split="train", streaming=True)
m = 5
top_m = list(itertools.islice(instruction_tuned_dataset, m))
for j in top_m:
    print(j)

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}
{'instruction': 'What are the three primary colors?', 'input': '', 'output': 'The three primary colors are red, blue, and yellow.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat are the three primary colors?\n\n### Respo

In [7]:
processed_data = []
prompt_template_without_input = '''Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:'''
prompt_template_with_input = ('''
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:''')
for j in top_m:
    if j["input"]:
        processed_prompt = prompt_template_with_input.format(instruction=j["instruction"], input=j["input"])
    else:
        processed_prompt = prompt_template_without_input.format(instruction=j["instruction"])
    processed_data.append({"input": processed_prompt, "output": j["output"]})
print(processed_data[0])

{'input': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


In [8]:
with jsonlines.open("alpaca_processed.jsonl", "w") as writer:
    writer.write_all(processed_data)

In [9]:
dataset_hf = load_dataset("lamini/alpaca")
print(dataset_hf)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 52002
    })
})


In [10]:

llm = Lamini("meta-llama/Llama-3.2-3B-Instruct")
print(
    llm.generate(
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Which animal remembers facts the best?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    )
)




While animals are known for their impressive memory abilities, some species stand out for their exceptional memory capabilities. Here are some of the top contenders:

1. **Chimpanzees**: Our closest living relatives in the animal kingdom, chimpanzees have been observed to possess excellent memory. They can recall:
	* Complex social hierarchies and relationships
	* Tool use and usage
	* Food sources and locations
	* Even human faces and interactions
2. **Dolphins**: These intelligent marine mammals have been known to:
	* Recognize and remember individual humans, including their voices and faces
	* Recall complex social behaviors and relationships
	* Learn and perform tricks and tasks
	* Even recognize themselves in mirrors
3. **Elephants**: With their large brains and complex social structures, elephants have impressive memory abilities:
	* They can remember:
		+ Family members and social relationships
		+ Migration routes and habitats
		+ Water sources and food locations
		+ Even the

In [11]:

llm = Lamini("meta-llama/Llama-2-7b-chat-hf")
print(
    llm.generate(
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Which animal remembers facts the best?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
    )
)

I'm just an AI, I don't have personal opinions or beliefs, but I can provide you with some interesting facts about animals and their memory abilities.

Some animals are known to have exceptional memory abilities, including:

1. Elephants: Elephants have been observed to have excellent memory skills, particularly when it comes to remembering their herd members, their social hierarchy, and their territory. They have been known to recognize and greet old friends and family members after many years of separation.
2. Dolphins: Dolphins are known to have excellent memory and cognitive abilities, including the ability to recognize and remember individual humans. They have been observed to remember and respond to complex commands and tasks after many years of training.
3. Octopuses: Octopuses have been found to have excellent memory skills, particularly when it comes to remembering complex patterns and tasks. They have been observed to solve problems and complete tasks after many years of trai

In [13]:
# chatgpt = Lamini("chat-gpt")
# instruct_output_chatgpt = chatgpt.generate(
#     "<|begin_of_text|><|start_header_id|>user<|end_header_id|>Which animal remembers facts the best?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
# )
# print(instruct_output_chatgpt)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-70m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/pythia-70m")

In [6]:
def inference(text, model, tokenizer, max_input_tokens=1000,max_output_token=1000):
    input_ids = tokenizer.encode(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_tokens
    )
    device = model.device
    generated_tokens_with_prompt = model.generate(
        input_ids=input_ids.to(device),
        max_length=max_output_tokens
    )

    generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt)

    generated_text_answer = generated_text_with_prompt[0][len(text):]

    return generated_text_answer
    